In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.externals import joblib
import re
from sklearn import metrics as m
from IPython.display import clear_output
import timeit

Using TensorFlow backend.


In [41]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
features = pd.read_csv('../datasets/features.csv')
features.rename(index=str, inplace=True, columns={
    'srcip': 'ipv4_src',
    'sport': 'port_src',
    'dstip': 'ipv4_dst',
    'dsport': 'port_dst'})
real = pd.read_csv('../datasets/UNSW-NB15_1.csv', names=features)
# 120 000 records are the all records from first pcap
real = real.head(120000)
real[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']] = real[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']].astype(str)

/home/jozef/anaconda2/lib/python2.7/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)
/home/jozef/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
real.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0,0,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0,0,0,7,9,1,1,1,1,1,NaN,0


In [5]:
real.shape

(120000, 49)

In [43]:
flows = pd.read_csv('../results/22-01-2015_1_200000_pcap0_w10.dump', delimiter=',')
flows['real_label'] = 0

In [44]:
flows.shape

(2260, 15)

In [45]:
packets_count = flows[['packets_src','packets_dst']].sum()
packets_count.sum()

139402

In [46]:
flows.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,prediction,category,probability,real_label
0,175.45.176.0,13284,149.171.126.16,80,6,846,124,8,2,1,1,1,Warning,76.46%,0
1,59.166.0.5,3593,149.171.126.5,53,17,160,192,2,2,3,5,0,Normal,0.01%,0
2,59.166.0.3,49664,149.171.126.0,53,17,174,206,2,2,1,1,0,Normal,0.11%,0
3,59.166.0.5,6645,149.171.126.2,80,6,272,140,4,2,1,1,0,Normal,31.73%,0
4,59.166.0.3,42587,149.171.126.8,25,6,272,140,4,2,1,2,0,Normal,31.38%,0


In [26]:
flows.loc[(flows['category'] == 'Warning') | (flows['category'] == 'Best_effort') | (flows['category'] == 'Attack')]

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,prediction,category,probability,real_label
0,175.45.176.0,13284,149.171.126.16,80,6,1558,352,14,6,2,3,1,Warning,62.09%,0
9,59.166.0.0,2109,149.171.126.9,52908,6,404,140,6,2,1,17,1,Warning,62.85%,0
18,59.166.0.9,42988,149.171.126.8,48017,6,404,140,6,2,1,15,1,Warning,62.29%,0
31,59.166.0.7,18282,149.171.126.9,5190,6,404,140,6,2,1,17,1,Warning,62.85%,0
41,175.45.176.2,23357,149.171.126.16,80,6,1086,25888,12,24,2,3,1,Attack,97.72%,0
46,59.166.0.6,5544,149.171.126.4,22,6,404,140,6,2,1,14,1,Warning,61.95%,0
79,175.45.176.2,26939,149.171.126.10,80,6,984,352,10,6,1,1,1,Best_effort,84.13%,0
87,59.166.0.0,3507,149.171.126.3,22,6,404,140,6,2,1,13,1,Warning,61.61%,0
90,59.166.0.4,8987,149.171.126.9,13883,6,404,140,6,2,1,17,1,Warning,62.85%,0
99,175.45.176.0,39500,149.171.126.15,80,6,1354,352,10,6,2,2,1,Warning,60.07%,0


In [47]:
flows[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']] = flows[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']].astype(str)
start = timeit.default_timer()
for i in flows.index:
    clear_output(wait=True)

    tmp = real['Label'].loc[((real['ipv4_src'] == flows['ipv4_src'][i]) & 
               (real['port_src'] == flows['port_src'][i]) &
               (real['ipv4_dst'] == flows['ipv4_dst'][i]) &
               (real['port_dst'] == flows['port_dst'][i])) | (
               (real['ipv4_src'] == flows['ipv4_dst'][i]) & 
               (real['port_src'] == flows['port_dst'][i]) &
               (real['ipv4_dst'] == flows['ipv4_src'][i]) &
               (real['port_dst'] == flows['port_src'][i]))]
    flows['real_label'][i] = tmp.values[0]

    stop = timeit.default_timer()
    if (float(float(i + 1) / float(len(flows.index)) * 100) < 5):
        expected_time = 'Calculating...'
    else:
        time_perc = timeit.default_timer()
        expected_time = np.round((float(time_perc - start) / (float(i + 1) / float(len(flows.index)))) / 60,2)
    print 'Progress: ' + str(i+1) + '/' + str(len(flows.index))
    print 'Percentage: ' + str(np.round(float(float(i + 1) / float(len(flows.index))) * 100, 2)) + '%'
    print 'Run time: ' + str(np.round((stop - start) / 60, 2)) + ' minutes'
    print 'Expected run time: ' + str(expected_time) + ' minutes'

KeyboardInterrupt: 

In [ ]:
custom = flows.replace(to_replace='Warning', value='Attack')
custom = custom.replace(to_replace='Best_effort', value='Attack')
evaluation = custom[['ipv4_src', 'category', 'real_label']]
evaluation['ipv4_src'] = evaluation.apply(lambda row: 'Malicious user' if row['real_label'] == 1 else 'Normal user', axis=1)
evaluation = evaluation[['ipv4_src', 'category']]
evaluation['count'] = 1
evaluation = evaluation.groupby(['ipv4_src', 'category']).count()
evaluation

In [ ]:
e = evaluation.unstack(fill_value=0)
e
try:
    TP = e.loc['Malicious user'].loc['count'].loc['Attack']
except:
    TP = 0
try:
    TN = e.loc['Normal user'].loc['count'].loc['Normal']
except:
    TN = 0
try:
    FP = e.loc['Normal user'].loc['count'].loc['Attack']
except:
    FP = 0
try:
    FN = e.loc['Malicious user'].loc['count'].loc['Normal']
except:
    FN = 0
print 'TP:', TP
print 'TN:', TN
print 'FP:', FP
print 'FN:', FN
try:
    acc = float(TP + TN)/float(TP + TN + FP + FN)
except:
    acc = 0
print ('Accuracy is %.2f%%' % (acc*100))
try:
    sen = float(TP)/float(TP + FN)
except:
    sen = 0
print ('Sensitivity is %.2f%%' % (sen*100))
try:
    spe = float(TN)/float(TN + FP)
except:
    spe = 0
print ('Specificity is %.2f%%' % (spe*100))
try:
    pre = float(TP)/float(TP + FP)
except:
    pre = 0
print ('Precision is %.2f%%' % (pre*100))
try:
    f1_score = 2*(float(pre*sen)/float(pre+sen))
except:
    f1_score = 0
print ('F1 score is %.2f%%' % (f1_score*100))

In [46]:
packets_count = flows[['packets_src','packets_dst']].sum()
packets_count.sum()

19444

# Model evaluation, different test

In [58]:
model = load_model('../models/DNN_model_all_binary_full.h5')
scaler = joblib.load('../models/DNN_model_all_binary_full_scaler.sav')

/home/jozef/anaconda2/lib/python2.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [150]:
def switch_values(flow):
    if re.match(r'^149.*', flow['ipv4_src']):
        tmp_ip = flow['ipv4_src']
        tmp_port = flow['port_src']
        tmp_bytes = flow['bytes_src']
        tmp_pkts = flow['packets_src']
        flow['ipv4_src'] = flow['ipv4_dst']
        flow['port_src'] = flow['port_dst']
        flow['ipv4_dst'] = tmp_ip
        flow['port_dst'] = tmp_port
        flow['bytes_src'] = flow['bytes_dst']
        flow['packets_src'] = flow['packets_dst']
        flow['bytes_dst'] = tmp_bytes
        flow['packets_dst'] = tmp_pkts
    return flow            

flow = flows.loc[flows['real_label'] == 1]
# flow = flow.apply(lambda x: switch_values(x), axis=1)
columns = flow.loc[:, 'packets_src':'packets_dst']
columns = pd.concat([columns, flow.loc[:, 'bytes_src':'bytes_dst']], axis=1)
columns = pd.concat([columns, flow.loc[:, 'srv_dst_count':'dst_count']], axis=1)
columns['proto_arp'] = columns['proto_icmp'] = columns['proto_igmp'] = columns['proto_ospf'] = columns['proto_other'] = columns['proto_tcp'] = columns['proto_udp'] = 0
for i in range(len(columns.index)):
    if flow.loc[columns.index[i]]['proto'] == -1:
        columns.loc[columns.index[i]]['proto_arp'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 1:
        columns.loc[columns.index[i]]['proto_icmp'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 2:
        columns.loc[columns.index[i]]['proto_igmp'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 89:
        columns.loc[columns.index[i]]['proto_ospf'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 6:
        columns.loc[columns.index[i]]['proto_tcp'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 17:
        columns.loc[columns.index[i]]['proto_udp'] = 1
    else:
        columns.loc[columns.index[i]]['proto_other'] = 1
columns['packets_src'] = 10
columns['packets_dst'] = 10
columns['bytes_src'] = 842
columns

,packets_src,packets_dst,bytes_src,bytes_dst,srv_dst_count,dst_count,proto_arp,proto_icmp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
78,10,10,842,240,1,1,0,0,0,0,0,1,0


In [151]:
model.predict_proba(scaler.transform(columns))

/home/jozef/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


array([[0.76853204]], dtype=float32)

In [107]:
test = real.loc[(real['ipv4_src'] == flow.loc[78]['ipv4_src']) & 
               (real['port_src'] == flow.loc[78]['port_src']) &
               (real['ipv4_dst'] == flow.loc[78]['ipv4_dst']) &
               (real['port_dst'] == flow.loc[78]['port_dst'])]

In [112]:
test

,ipv4_src,port_src,ipv4_dst,port_dst,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
365,175.45.176.1,13377,149.171.126.12,80,tcp,FIN,0.187885,842,5084,62,252,2,3,http,32275.0625,194842.5938,10,10,255,255,114736493,263943961,84,508,1,2076,1103.263956,1668.959802,1421927443,1421927443,20.876111,19.461111,0.054718,0.009946,0.044772,0,1,1,0,0,1,1,1,1,1,1,1,Exploits,1


In [120]:
sample = test.loc[:, 'Spkts':'Dpkts']
sample = pd.concat([sample, test.loc[:, 'sbytes':'dbytes']], axis=1)
sample = pd.concat([sample, test.loc[:, 'ct_srv_dst':'ct_dst_ltm']], axis=1)
sample['proto_arp'] = sample['proto_icmp'] = sample['proto_igmp'] = sample['proto_ospf'] = sample['proto_other'] = sample['proto_tcp'] = sample['proto_udp'] = 0
i = 365
if test.loc[i]['proto'] == 'arp':
    sample['proto_arp'] = 1
elif test.loc[i]['proto'] == 'icmp':
    sample['proto_icmp'] = 1
elif test.loc[i]['proto'] == 'igmp':
    sample['proto_igmp'] = 1
elif test.loc[i]['proto'] == 'ospf':
    sample['proto_ospf'] = 1
elif test.loc[i]['proto'] == 'tcp':
    sample['proto_tcp'] = 1
elif test.loc[i]['proto'] == 'udp':
    sample['proto_udp'] = 1
else:
    sample['proto_other'] = 1
print (model.predict_proba(scaler.transform(sample)))

[[0.7120315]]


/home/jozef/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


In [128]:
columns

,packets_src,packets_dst,bytes_src,bytes_dst,srv_dst_count,dst_count,proto_arp,proto_icmp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
78,4,8,842,5112,1,1,0,0,0,0,0,1,0


In [129]:
sample

,Spkts,Dpkts,sbytes,dbytes,ct_srv_dst,ct_dst_ltm,proto_arp,proto_icmp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
365,10,10,842,5084,1,1,0,0,0,0,0,1,0


In [191]:
df = pd.DataFrame([[acc, sen, spe, pre]], columns=means_names)
means = means.append(df)

In [192]:
means

,Accuracy,Sensitivity,Specificity,Precision
0,0.750000,1.000000,0.727273,0.250000
0,0.807692,1.000000,0.791667,0.285714
0,0.761905,0.666667,0.777778,0.333333
0,0.741935,0.428571,0.833333,0.428571
0,0.769231,0.800000,0.761905,0.444444
0,0.780000,0.750000,0.782609,0.230769
0,0.702128,0.600000,0.714286,0.200000
0,0.977273,0.888889,0.983740,0.800000
0,0.975410,1.000000,0.973684,0.727273


In [193]:
means['Accuracy'].mean()

0.8072859495123974

In [194]:
means['Sensitivity'].mean()

0.7926807760141092

In [195]:
means['Specificity'].mean()

0.8162526360908716

In [196]:
means['Precision'].mean()

0.4111228277894945

## Notes and unused commands

In [6]:
means_names = ['Accuracy','Sensitivity','Specificity','Precision']
means = pd.DataFrame(columns=means_names)
means

,Accuracy,Sensitivity,Specificity,Precision


In [ ]:
gr_data = flows[['ipv4_src', 'category']]
gr_data['count'] = 1
gr_data = gr_data.groupby(['ipv4_src', 'category']).count()
gr_data

In [172]:
flows.loc[flows['category'] == 'Warning']

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,prediction,category,probability,real_label
13,59.166.0.6,11527,149.171.126.2,22437,6,878,736,10,8,1,5,[1],Warning,66.03%,0
27,59.166.0.4,40650,149.171.126.7,38455,6,1394,1142,16,14,1,3,[1],Warning,60.19%,0
32,59.166.0.9,42988,149.171.126.8,48017,6,1054,802,12,10,1,9,[1],Warning,67.02%,0


In [126]:
flows = whole.loc[169:]
flows.reset_index(drop=True, inplace=True)
flows.shape

(162, 15)

In [120]:
whole = flows